In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import random
import math
import warnings
import scipy as sp

# Read the image
image_original = cv2.imread('ICIP training data\class0\RawDataQA (6)\RawDataQA6 (1).tiff', cv2.IMREAD_GRAYSCALE)
image = np.zeros((len(image_original), len(image_original[0])))

for row in range(len(image)):
  for pixel in range(len(image[row])):
    image[row][pixel] = np.log(image_original[row][pixel]+1)
   
# for row in range(len(image)):
#   for pixel in range(len(image[row])):
#     print(image[row][pixel])
     
image = image.flatten()
global num_pixels , num_components, sigma_noise, maxi
maxi = [0]
num_pixels = len(image)
num_components = 5

In [3]:
# def cum_normal_laplace(y, mean, sigma):
#   factor1 = 0.5
#   factor2 = sp.special.erf((y-mean)/(math.sqrt(2)*sigma_noise))
#   factor3 = -np.exp(sigma_noise**2/sigma**2 - math.sqrt(2)*(y-mean)/sigma)
#   factor4 = 0.5*np.exp(-((y-mean)/(math.sqrt(2)*sigma_noise))**2)
#   factor5 = sp.special.erfcx((y-mean)/(math.sqrt(2)*sigma_noise)+sigma_noise/sigma)
#   factor6 = sp.special.erfcx((y-mean)/(math.sqrt(2)*sigma_noise)-sigma_noise/sigma)
#   print(factor1, factor2, factor3, factor4, factor5, factor6)
#   return factor1*(1+factor2+factor3+factor4*(factor5+factor6))

def cum_normal_laplace(y, mean, sigma):
  alpha = math.sqrt(2)/sigma
  beta = alpha
  factor1 = 1/(2*(alpha+beta))
  factor2 = 2*np.exp(0.5*alpha*(-2*y+2*mean+alpha*sigma_noise**2))*beta
  factor3 = (alpha+beta)*sp.special.erf((y-mean)/(math.sqrt(2)*sigma_noise))
  factor4 = np.exp(0.5*alpha*(-2*y+2*mean+alpha*sigma_noise**2))*beta*sp.special.erfc(-alpha*sigma_noise/math.sqrt(2)+(y-mean)/(math.sqrt(2)*sigma_noise))
  factor5 = np.exp(0.5*beta*(2*y-2*mean+beta*sigma_noise**2))*alpha*sp.special.erfc(beta*sigma_noise/math.sqrt(2)+(y-mean)/(math.sqrt(2)*sigma_noise))
  return factor1*(alpha+beta - factor2 + factor3+factor4+factor5)
def find_cum_normal_laplace(image, mean_components, sigma_components):
  cum_components = []
  for index in range(num_components):
    cum_component = []
    for pixel in range(num_pixels):
      cum_component.append(cum_normal_laplace(image[pixel], mean_components[index], sigma_components[index]))
    cum_components.append(cum_component)
  return cum_components
    
    


def normal_laplace(image,mean, sigma):
  global maxi
  function_y = []
  for i in range(num_pixels):
    alpha = math.sqrt(2)/sigma
    beta = alpha
    y= image[i]
    factor1 = alpha*beta/(2*(alpha+beta))
          
    factor2 = (np.exp(0.5*alpha*(-2*y+2*mean+alpha*sigma_noise**2)))*sp.special.erfc(alpha*sigma_noise/math.sqrt(2)-(y-mean)/(math.sqrt(2)*sigma_noise))
    
    factor3 = (np.exp(0.5*beta*(2*y-2*mean+beta*sigma_noise**2)))*sp.special.erfc(beta*sigma_noise/math.sqrt(2)+(y-mean)/(math.sqrt(2)*sigma_noise))
    # factor1 =  -1/(2*(math.sqrt(2)*sigma))
    # factor2 = (np.exp(sigma_noise**2/sigma**2 + math.sqrt(2)*(y-mean)/sigma))*math.erf(-(y-mean)/(math.sqrt(2)*sigma_noise) - sigma_noise/sigma)
    # factor3 = (np.exp(sigma_noise**2/sigma**2 - math.sqrt(2)*(y-mean)/sigma))*math.erf(-(y-mean)/(math.sqrt(2)*sigma_noise) + sigma_noise/sigma)
    
    #This works
    # factor1 = alpha*beta/(2*(alpha+beta))*np.exp(-(y-mean)**2/(2*sigma_noise**2))*0.5
    # factor2 = math.erf((y-mean)/(math.sqrt(2)*sigma_noise)+sigma_noise*beta/math.sqrt(2)) 
    # factor3 = math.erf(-(y-mean)/(math.sqrt(2)*sigma_noise)+sigma_noise*alpha/math.sqrt(2)) 
    
    result = factor1*(factor2+factor3)
  
    #print(y, mean, factor1,factor2, factor3)
    function_y.append(result)
    
  return function_y


def initialize_f_y(image, mean_components, sigma_components):
  new_f_y_components = []
  for index in range(num_components):
    new_f_y_components.append(normal_laplace(image,mean_components[index],sigma_components[index]))
  return new_f_y_components

def update_r( f_y, a_components):
  new_r_components = []
  for component_index in range(num_components):
    new_r_component =[]
    for pixel in range(num_pixels):
      
      
      numerator = a_components[component_index]*f_y[component_index][pixel]
      denominator = 0 
      for i in range(num_components):#for the number of components
        denominator+= a_components[i]*f_y[i][pixel]
      if denominator == 0:
        denominator = 0.0001
      new_r_component.append(numerator/denominator)
    new_r_components.append(new_r_component)
  return new_r_components


def update_a(a_components, r_components):
  new_a_components = []
  for a_index in range(num_components):
    new_a_components.append(np.sum(r_components[a_index])/num_pixels)
    
  return new_a_components
    
def update_mean(mean_components,y, r_components ):
  new_mean_components = []
  for mean_index in range(num_components):
    numerator = 0
    denominator = 0
    for pixel in range(num_pixels):
      numerator+=y[pixel]*r_components[mean_index][pixel]
      denominator+=r_components[mean_index][pixel]
    if denominator == 0:
      new_mean_components.append(mean_components[mean_index])
    else:
      new_mean_components.append(numerator/denominator)
  return new_mean_components



def update_sigma(sigma_components,mean_components, y, r_components):
  new_sigma_components = []
  for sigma_index in range(num_components):
    numerator = 0
    denominator = 0
    for pixel in range(num_pixels):
      numerator+= ((y[pixel]-mean_components[sigma_index])**2)*r_components[sigma_index][pixel]
      denominator+= r_components[sigma_index][pixel]
    new_sigma_components.append( math.sqrt(max(numerator/denominator, 0.0001)) )
  return new_sigma_components
    
  

In [4]:
def diff_lt_delta(x1, x2, delta =0.02):
  if len(x1)!= len(x2):
    print("error")
  for i in range(len(x1)):
    if abs(x1[i]-x2[i])>delta:
      # print( abs(x1[i]-x2[i]))
      return False
  return True


In [5]:
##initializing the values for a, mu, and sigma
mean_img, sigma_img = cv2.meanStdDev(image)
mean_img = mean_img.flatten()[0]
sigma_img = sigma_img.flatten()[0]
mean_components = []
sigma_components = []
a_components = []
f_y_components = []
sigma_noise = 0.1

##initialization
for k in range(num_components):
  random_float = random.uniform(0.8, 1.2)
  mean_components.append(random_float*mean_img)
  sigma_components.append(random_float*sigma_img)
  a_components.append(1/num_components)

print(" a components are ", a_components)
print("sigma_components are ", sigma_components)

new_f_y_components = initialize_f_y(image, mean_components, sigma_components)
new_r = update_r(new_f_y_components,a_components)
new_a_components = update_a(a_components, new_r)
new_mean_components = update_mean(mean_components, image, new_r)
new_sigma_components = update_sigma(sigma_components, new_mean_components, image, new_r)

print(" a components are ", new_a_components)
print("sigma_components are ", new_sigma_components)

CONTINUE = (diff_lt_delta(sigma_components, new_sigma_components)==False or diff_lt_delta(new_a_components, a_components)==False) == True
while CONTINUE:
  
  
  mean_components = new_mean_components
  sigma_components = new_sigma_components
  a_components = new_a_components
  
  new_f_y_components = initialize_f_y(image, mean_components, sigma_components)
  new_r = update_r(new_f_y_components,a_components)
  new_a_components = update_a(a_components, new_r)
  new_mean_components = update_mean(mean_components, image, new_r)
  new_sigma_components = update_sigma(sigma_components, new_mean_components, image, new_r)
  print(" a components are ", new_a_components)
  print("mean_components are ", new_mean_components)
  print("sigma_components are ", new_sigma_components)
  CONTINUE = (diff_lt_delta(sigma_components, new_sigma_components)==False or diff_lt_delta(new_a_components, a_components)==False) == True

  

 a components are  [0.2, 0.2, 0.2, 0.2, 0.2]
sigma_components are  [0.24278548338312778, 0.1935435425426999, 0.21670117415635756, 0.2408762897252272, 0.22474377145941415]
 a components are  [0.018944287374313418, 0.35943999626128353, 0.4200925159523202, 0.023953268253771844, 0.17756993215831102]
sigma_components are  [0.20108259708575477, 0.18890467622389484, 0.1335297141678697, 0.1989564075400375, 0.15909912139966936]
0.04170288629737301
 a components are  [0.01793033444879633, 0.339535627927372, 0.44214852235616625, 0.022680987912160572, 0.17770452735550488]
mean_components are  [4.8491779536363655, 4.581985183682765, 4.794125971778015, 4.850559164184053, 4.831951553344228]
sigma_components are  [0.19091452834804712, 0.21358849850335057, 0.14633574328296084, 0.18896957506080939, 0.1622308823692363]
0.02468382227945573
 a components are  [0.017581682218593506, 0.32298835146697985, 0.45777334953896337, 0.022251652423478905, 0.17940496435198447]
mean_components are  [4.8243575972414225,

In [6]:
cum_components = find_cum_normal_laplace(image, new_mean_components, new_sigma_components)

In [7]:
def gaussanize( cum_components, mean_components, sigma_components):
  sigma_gauss = 1.5
  mean_gauss = -3
  gaussanized_components = []
  for index in range(num_components):
    gaussanized_component = []
    for pixel in range(num_pixels):
      gaussanized_component.append(mean_components[index]+mean_gauss+math.sqrt(2)*sigma_gauss*sp.special.erfinv(2*cum_components[index][pixel]-1))
    gaussanized_components.append(gaussanized_component)
  return gaussanized_components

def new_image(f_y_components, gaussanized_components, a_components ):
  new_image = []
  for pixel in range(num_pixels):
    numerator = 0
    denominator = 0
    for index in range(num_components):
      numerator+= a_components[index]*f_y_components[index][pixel]*gaussanized_components[index][pixel]
      denominator+= a_components[index]*f_y_components[index][pixel]
    new_image.append(numerator/denominator)
  return new_image

gaussanized_components = gaussanize(cum_components, new_mean_components, new_sigma_components)

new_gaussanized_image = new_image(new_f_y_components, gaussanized_components, new_a_components)

for i in range(num_pixels):
  new_gaussanized_image[i] = math.exp(new_gaussanized_image[i])-1
  
num_rows = len(image_original)
num_cols = len(new_gaussanized_image) // num_rows
new_image_view = np.array(new_gaussanized_image).reshape(num_rows, num_cols)

# Convert the pixel array to an image
new_image_view_int_cv = np.uint8(new_image_view)
new_image_view_cv = cv2.cvtColor(new_image_view_int_cv, cv2.COLOR_GRAY2BGR)  # Convert RGB to BGR for OpenCV

# Display the image using OpenCV
cv2.imshow('Image',new_image_view_cv)
cv2.imwrite('grayscale_image.png', new_image_view_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()
